In [1]:
!pip install confluent_kafka sseclient

  Using cached confluent_kafka-2.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached sseclient-0.0.27-py3-none-any.whl
Using cached confluent_kafka-2.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.1 MB)


In [2]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random

brokers = "kafka1:9092,kafka2:9093"

In [3]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [4]:
p = SerializingProducer(pconf)

In [5]:
stocks = "sample.csv"

In [9]:
import time
from datetime import datetime

def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") # Format to ISO 8601

    stock = {
        "pickup_datetime": row[5],  # Assuming pickup_datetime is at index 5
        "trip_distance": float(row[9]),  # Assuming trip_distance is at index 9
        "timestamp": str_date_time  # Current timestamp
    }
    return stock

# Example usage:
row = ["89D227B655E5C82AECF13C3F540D4CF4", "BA96DE419E711691B9445D6A6307C170", "CMT", "1", "N", "2013-01-01 15:11:48", "2013-01-01 15:18:10", "4", "382", "1.00", "-73.978165", "40.757977", "-73.989838", "40.751171"]
stock_data = construct_stock(row)
print(stock_data)


{'pickup_datetime': '2013-01-01 15:11:48', 'trip_distance': 1.0, 'timestamp': '2024-05-28T11:32:47Z'}


In [7]:
import csv
import json
from datetime import datetime

def construct_stock(row):
    try:
        return {
            "license": row[0],  
            "total_amount": float(row[1]),
            "timestamp": datetime.fromtimestamp(int(row[5])).strftime("%Y-%m-%dT%H:%M:%SZ")
        }
    except ValueError:
        print(f"Skipping row with non-numeric value: {row}")
        return None
  
stocks = "sample.csv"  
n = 0

with open(stocks) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    header = next(csv_reader) # read the header row
    for row in csv_reader:
        n = n + 1
        stock = construct_stock(row)
        if stock is not None:
            p.produce('stock', value=json.dumps(stock))
    

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Use only if you need to purge all the messages in the queue

In [8]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=['stock'])

{'stock': <Future at 0x7fa2120ba590 state=running>}